необходимо прикрепить к ноутбуку датасеты

https://www.kaggle.com/datasets/2ff46dd8ee67d3d36367196da5d381ebfbf5ebb1b76d3049b0b1a7ff397d7fac

https://www.kaggle.com/datasets/b39d095daccd2e7e79e80abdd278948d4c90a4d775b6b3d2df9bbc2766f137ce

Создание аугментированной версии датасетов, на примере youtube_control_001

In [ ]:
!pip install datasets==2.10.1
!pip install transformers==4.27.3
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install audiomentations

In [ ]:
import pandas as pd
import evaluate
import jiwer
import os
import shutil
from datasets import load_dataset
from datasets import Audio
from huggingface_hub import login
import torch
from IPython import display
from matplotlib import pyplot as plt
from audiomentations import Compose, AddGaussianNoise, AddBackgroundNoise, PolarityInversion, ApplyImpulseResponse
from datasets import interleave_datasets

In [ ]:
login(token='') # здесь должен быть write токен аккаунта, на котором создается датасет

In [4]:
def visualize_audio(wav: torch.Tensor, sr: int = 16000):
    # Average all channels
    if wav.dim() == 2:
        # Any to mono audio convertion
        wav = wav.mean(dim=0)
    
    plt.figure(figsize=(20, 5))
    plt.plot(wav, alpha=.7, c='green')
    plt.grid()
    plt.xlabel('Time', size=20)
    plt.ylabel('Amplitude', size=20)
    plt.show()
    
    display.display(display.Audio(wav, rate=sr))

In [5]:
DATASET_NAME = 'aanosov/youtube_control_001'

In [ ]:
dataset = load_dataset(DATASET_NAME, split='train', use_auth_token=True)

In [7]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [10]:
augment_waveform = Compose([
    
    AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=0.2),
    
    ApplyImpulseResponse(ir_path='/kaggle/input/impulseresponse-dataset', p=0.2), 

    AddBackgroundNoise(
    sounds_path="/kaggle/input/esc50-noises-test",
    min_snr_in_db=3.0, max_snr_in_db=30.0,
    noise_transform=PolarityInversion(), p=0.2),
    
    ])


def augment_dataset(batch):
    
    audio = batch["audio"]["array"]

    augmented_audio = augment_waveform(samples=audio, sample_rate=16000)

    batch["audio"]["array"] = augmented_audio

    return batch

In [ ]:
dataset1 = dataset.map(augment_dataset)
dataset2 = dataset.map(augment_dataset)
dataset3 = dataset.map(augment_dataset)

In [12]:
dataset_aug = interleave_datasets([dataset1, dataset2, dataset3])

In [18]:
DATASET_NAME = f'{DATASET_NAME}_aug'

dataset_aug.push_to_hub(DATASET_NAME, private=True)

Map:   0%|          | 0/327 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]